## SOURCE  of This Notebook
https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/ 
#### Data file download - sign in required with registration
https://www.kaggle.com/sdolezel/black-friday
Once logged in, 'Download' link will be available around top-right corner
#### In the downloaded zip file, you will upload 'train' data file to Jupyter Notebook

### Columns
User_ID    User ID  
Product_ID      Product ID  
Gender     Sex of User  
Age   Age in bins  
Occupation      Occupation (Masked)  
City_Category   Category of the City (A,B,C)  
Stay_In_Current_City_Years     Number of years stay in current city  
Marital_Status Marital Status  
Product_Category_1  
Product_Category_2  
Product_Category_3  
Purchase   Purchase Amount 

#### Setup Apache Spark Context


In [1]:
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext('local[*]')
sqlContext = SQLContext(sc)

#### Creating the DataFrame from CSV file
#### This is where you import your own csv file

In [2]:
train = sqlContext.read.csv("train.csv", header = True, inferSchema = True)


#### How to see datatype of columns?


In [3]:
train.printSchema()


root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



#### How to Show first n observation? 
We can use head operation to see first n observation (say, 5 observation). Head operation in PySpark is similar to head operation in Pandas.

In [4]:
train.head(5)


[Row(User_ID=1000001, Product_ID='P00069042', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=3, Product_Category_2=None, Product_Category_3=None, Purchase=8370),
 Row(User_ID=1000001, Product_ID='P00248942', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=1, Product_Category_2=6, Product_Category_3=14, Purchase=15200),
 Row(User_ID=1000001, Product_ID='P00087842', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=None, Product_Category_3=None, Purchase=1422),
 Row(User_ID=1000001, Product_ID='P00085442', Gender='F', Age='0-17', Occupation=10, City_Category='A', Stay_In_Current_City_Years='2', Marital_Status=0, Product_Category_1=12, Product_Category_2=14, Product_Category_3=None, Purchase=1057),
 Row(User_ID=1000002, Product_ID='P0

Above results are comprised of row like format. To see the result in more interactive manner (rows under the columns), we can use the show operation. Let’s apply show operation on train and take first 2 rows of it. We can pass the argument truncate = True to truncate the result.

In [5]:
train.show(2,truncate= True)


+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

#### How to Count the number of rows in DataFrame?


In [6]:
train.count()


550068

#### How many columns do we have in train and test files along with their names?


In [7]:
len(train.columns), train.columns


(12,
 ['User_ID',
  'Product_ID',
  'Gender',
  'Age',
  'Occupation',
  'City_Category',
  'Stay_In_Current_City_Years',
  'Marital_Status',
  'Product_Category_1',
  'Product_Category_2',
  'Product_Category_3',
  'Purchase'])

#### How to get the summary statistics (mean, standard deviance, min ,max, count) of numerical columns in a DataFrame? 
describe operation is use to calculate the summary statistics of numerical column(s) in DataFrame. If we don’t specify the name of columns it will calculate summary statistics for all numerical columns present in DataFrame.

In [8]:
train.describe().show()


+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+-------------------+------------------+------------------+------------------+-----------------+
|  count|            550068|    550068|550068|550068|           550068|       550068|                    550068|             550068|            550068|            376430|            166821|           550068|
|   mean|1003028.8424013031|      null|  null|  null|8.076706879876669|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 9.84232925112238

In [9]:
train.describe('Product_ID').show()


+-------+----------+
|summary|Product_ID|
+-------+----------+
|  count|    550068|
|   mean|      null|
| stddev|      null|
|    min| P00000142|
|    max|  P0099942|
+-------+----------+



#### How to select column(s) from the DataFrame?


In [10]:
train.select('User_ID','Age').show(5)


+-------+----+
|User_ID| Age|
+-------+----+
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000001|0-17|
|1000002| 55+|
+-------+----+
only showing top 5 rows



#### What if I want to calculate pair wise frequency of categorical columns? 
We can use crosstab operation on DataFrame to calculate the pair wise frequency of columns. Let’s apply crosstab operation on ‘Age’ and ‘Gender’ columns of train DataFrame.

In [11]:
train.crosstab('Age', 'Gender').show()


+----------+-----+------+
|Age_Gender|    F|     M|
+----------+-----+------+
|      0-17| 5083| 10019|
|     46-50|13199| 32502|
|     18-25|24628| 75032|
|     36-45|27170| 82843|
|       55+| 5083| 16421|
|     51-55| 9894| 28607|
|     26-35|50752|168835|
+----------+-----+------+



#### What If I want to get the DataFrame which won’t have duplicate rows of given DataFrame?


In [12]:
train.select('Age','Gender').dropDuplicates().show()


+-----+------+
|  Age|Gender|
+-----+------+
|51-55|     F|
|18-25|     M|
| 0-17|     F|
|46-50|     M|
|18-25|     F|
|  55+|     M|
|  55+|     F|
|36-45|     M|
|26-35|     F|
| 0-17|     M|
|36-45|     F|
|51-55|     M|
|26-35|     M|
|46-50|     F|
+-----+------+



#### What if I want to drop the all rows with null value? 
The dropna operation can be use here. To drop row from the DataFrame it consider three options. 
* how– ‘any’ or ‘all’. If ‘any’, drop a row if it contains any nulls. If ‘all’, drop a row only if all its values are null.
* thresh – int, default None If specified, drop rows that have less than thresh non-null values. This overwrites the how parameter.
* subset – optional list of column names to consider. 

Let’t drop null rows in train with default parameters and count the rows in output DataFrame. Default options are any, None, None for how, thresh, subset respectively.


In [13]:
train.dropna().count()


166821

#### What if I want to fill the null values in DataFrame with constant number? 
Use fillna operation here. The fillna will take two parameters to fill the null values. 
+ value:
  + It will take a dictionary to specify which column will replace with which value.
  + A value (int , float, string) for all columns.
+ subset: Specify some selected columns.

Let’s fill ‘-1’ inplace of null values in train DataFrame.

In [14]:
train.fillna(-1).show(2)


+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|                -1|                -1|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
only

#### If I want to filter the rows in train which has Purchase more than 15000? 
We can apply the filter operation on Purchase column in train DataFrame to filter out the rows with values more than 15000. We need to pass a condition. Let’s apply filter on Purchase column in train DataFrame and print the number of rows which has more purchase than 15000.

In [15]:
train.filter(train.Purchase > 15000).count()


110523

#### How to find the mean of each age group in train? 
The groupby operation can be used here to find the mean of Purchase for each age group in train. Let’s see how can we get the mean purchase for the ‘Age’ column train.

In [16]:
train.groupby('Age').agg({'Purchase': 'mean'}).show()


+-----+-----------------+
|  Age|    avg(Purchase)|
+-----+-----------------+
|18-25|9169.663606261289|
|26-35|9252.690632869888|
| 0-17|8933.464640444974|
|46-50|9208.625697468327|
|51-55|9534.808030960236|
|36-45|9331.350694917874|
|  55+|9336.280459449405|
+-----+-----------------+



We can also apply sum, min, max, count with groupby when we want to get different summary insight each group. Let’s take one more example of groupby to count the number of rows in each Age group.

In [17]:
train.groupby('Age').count().show()


+-----+------+
|  Age| count|
+-----+------+
|18-25| 99660|
|26-35|219587|
| 0-17| 15102|
|46-50| 45701|
|51-55| 38501|
|36-45|110013|
|  55+| 21504|
+-----+------+



#### How to create a sample DataFrame from the base DataFrame? 
We can use sample operation to take sample of a DataFrame. The sample method on DataFrame will return a DataFrame containing the sample of base DataFrame. The sample method will take 3 parameters.

+ withReplacement = True or False to select a observation with or without replacement.
+ fraction = x, where x = .5 shows that we want to have 50% data in sample DataFrame.
+ seed for reproduce the result

Let’s create the two DataFrame t1 and t2 from train, both will have 20% sample of train and count the number of rows in each.

In [18]:
t1 = train.sample(False, 0.2, 42)
t2 = train.sample(False, 0.2, 43)
t1.count(),t2.count()

(109839, 109861)

#### How to apply map operation on DataFrame columns? 
We can apply a function on each row of DataFrame using map operation. After applying this function, we get the result in the form of RDD. Let’s apply a map operation on User_ID column of train and print the first 5 elements of mapped RDD(x,1) after applying the function (I am applying lambda function). 

Spark 2.0 (https://stackoverflow.com/questions/39535447/attributeerror-dataframe-object-has-no-attribute-map) 
You can't map a dataframe, but you can convert the dataframe to an RDD and map that by doing spark_df.rdd.map(). Prior to Spark 2.0, spark_df.map would alias to spark_df.rdd.map(). With Spark 2.0, you must explicitly call .rdd first.

Map vs Filter (https://stackoverflow.com/questions/40459695/map-vs-filter-operations)

Map, you pass in a function which returns a value for each element in an array. <strong>The return value of this function represents what an element becomes in our new array.</strong>

Filter, you pass in a function which returns either true or false for each element. <strong>If the function that you pass returns true for an element, then that element is included in the final array.</strong>

In [19]:
train.select('User_ID').rdd.map(lambda x:(x,1)).take(5)


[(Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000001), 1),
 (Row(User_ID=1000002), 1)]

#### How to sort the DataFrame based on column(s)? 
We can use orderBy operation on DataFrame to get sorted output based on some column. The orderBy operation take two arguments.

+ List of columns.
+ ascending = True or False for getting the results in ascending or descending order(list in case of more than two columns )

Let’s sort the train DataFrame based on ‘Purchase’.

In [20]:
train.orderBy(train.Purchase.desc()).show(5)


+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1003160| P00052842|     M|26-35|        17|            C|                         3|             0|                10|                15|              null|   23961|
|1002272| P00052842|     M|26-35|         0|            C|                         1|             0|                10|                15|              null|   23961|
|1001474| P00052842|     M|26-35|         4|            A|                         2|             1|                10|                15|              null|   23961

We can use withColumn operation to add new column (we can also replace) in base DataFrame and return a new DataFrame. The withColumn operation will take 2 parameters.

+ Column name which we want add /replace.
+ Expression on column.

Let’s see how withColumn works. I am calculating new column name ‘Purchase_new’ in train which is calculated by dviding Purchase column by 2.

In [21]:
train.withColumn('Purchase_new', train.Purchase /2.0).select('Purchase','Purchase_new').show(5)


+--------+------------+
|Purchase|Purchase_new|
+--------+------------+
|    8370|      4185.0|
|   15200|      7600.0|
|    1422|       711.0|
|    1057|       528.5|
|    7969|      3984.5|
+--------+------------+
only showing top 5 rows



#### How to Apply SQL Queries on DataFrame? 
We have already discussed in the above section that DataFrame has additional information about datatypes and names of columns associated with it. Unlike RDD, this additional information allows Spark to run SQL queries on DataFrame. To apply SQL queries on DataFrame first we need to register DataFrame as table. Let’s first register train DataFrame as table.

In [22]:
# NOTE. spark 2.0.0+
train.createOrReplaceTempView('train_table')

In [23]:
sqlContext.sql('select Product_ID from train_table').show(5)


+----------+
|Product_ID|
+----------+
| P00069042|
| P00248942|
| P00087842|
| P00085442|
| P00285442|
+----------+
only showing top 5 rows



Let’s get maximum purchase of each Age group in train_table.


In [24]:
sqlContext.sql('select Age, max(Purchase) from train_table group by Age').show()


+-----+-------------+
|  Age|max(Purchase)|
+-----+-------------+
|18-25|        23958|
|26-35|        23961|
| 0-17|        23955|
|46-50|        23960|
|51-55|        23960|
|36-45|        23960|
|  55+|        23960|
+-----+-------------+



## SOURCE  of This Notebook
https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/ 